In [35]:
# Librerias
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

In [12]:
RNG_SEED = 1922
np.random.seed(seed=RNG_SEED)

In [6]:
ceip_filepath = os.path.join("..", "data", "df_ceip.csv")

In [7]:
# Cargamos nuestro dataset con únicamente las columnas necesarias para predecir (según las conclusiones del EDA)
df = pd.read_csv(ceip_filepath, encoding="utf-8", usecols=["cod_evaluacion_final", "cod_motivo_repeticion", "cod_concepto_final_cond"]) \
.astype(
    {
        "cod_evaluacion_final":"category",
        "cod_motivo_repeticion":"category",
        "cod_concepto_final_cond":"category",
    }
) \
.dropna()

In [39]:
# Variables independientes (X) y variable dependiente (y)
X, y = df.drop("cod_evaluacion_final", axis=1), df["cod_evaluacion_final"]

In [40]:
# Estrategia de data splitting para evitar data leakage
kf = KFold(n_splits=5, shuffle=True)

In [41]:
# Modelo
model = RandomForestClassifier()

In [43]:
# Búsqueda de hiperparametros
param_grid = {
    'n_estimators': [5*i for i in range(2, 11, 2)], # más allá de 50 estimadores se vuelve bastante demandante computacionalmente
    'max_depth': [2, 5, 7] # idem, solo que elegidos al pseudo-azar en orden ascendente
}
grid_clf = GridSearchCV(model, param_grid, cv=kf)

In [44]:
# training y testing sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

In [45]:
# Entrenar el modelo
grid_clf.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 5, 7],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [55]:
# Obtener el mejor modelo y calculamos las predicciones sobre el test set
best_model = grid_clf.best_estimator_
predicciones = best_model.predict(X_test)

In [64]:
# Calculamos métricas
accuracy = best_model.score(X_test, y_test)
print(f"Precisión: {accuracy:.4f}")
print(f"Modelo classification report:\n {classification_report(y_test, predicciones)}")

Precisión: 0.9960


C:\Users\Gonzaa\anaconda3\envs\utec\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Modelo:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        32
         1.0       1.00      1.00      1.00    573310
         2.0       0.91      0.99      0.95     22941

    accuracy                           1.00    596283
   macro avg       0.64      0.66      0.65    596283
weighted avg       1.00      1.00      1.00    596283



C:\Users\Gonzaa\anaconda3\envs\utec\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Gonzaa\anaconda3\envs\utec\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [96]:
# Predecimos el dataset de validación
verify_df = pd.read_csv("../data/st_verify_data.csv")
df_solucion = pd.concat(
    [
        verify_df[["id"]],
        pd.Series(best_model.predict(verify_df[["cod_motivo_repeticion", "cod_concepto_final_cond"]])).to_frame("evaluacion_final")
    ],
    axis=1
)

In [97]:
# Escribimos el resultado
df_solucion.to_csv("entrega_final_gonzalo_ruiz_lti_minas.csv")